In [8]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras
import numpy as np
from pathlib import Path
import time
start_time = time.time()
from tensorflow.keras.utils import get_file
from tensorflow.keras.optimizers import RMSprop

# Needed the following as caused CUDA DNN errors
physical_devices = tf.config.list_physical_devices('GPU')
path = get_file('alice_in_wonderland.txt', origin='https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))

print('Unique characters:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)
print('Vectorization....')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# Build the model
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))


optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

import random
import sys
epochiteration = 1
inum = 1
inumlast = 21
for epoch in range(1, 5):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen -1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    epochiteration = 0

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)
        print("start text")
        sys.stdout.write(generated_text)
        print("end text")
        epochiteration += 1
        print("**************Loop write start*************")
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose = 0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)
        print("**************Loop write end**************")

        if inum < inumlast and epochiteration == 2:
            print("writing the following to file")
            
print("Complete: --- %s seconds has passed ---" % (time.time() - start_time))

Corpus length: 148574
Number of sequences: 49505
Unique characters: 46
Vectorization....


C:\Users\KDOSSE~1\AppData\Local\Temp/ipykernel_28412/1647475915.py:40: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
C:\Users\KDOSSE~1\AppData\Local\Temp/ipykernel_28412/1647475915.py:41: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=n

epoch 1
Train on 49505 samples
49505/49505 [==============================] - 35s 713us/sample - loss: 2.2174
--- Generating with seed: "unished for it now, i
suppose, by being drowned in my own te"
----- temperature: 0.2
start text
unished for it now, i
suppose, by being drowned in my own teend text
**************Loop write start*************
re the ding the catter her the was the said the sing the said the was the said the said the said the had she with the said the said the said the said the care the hat she had the had the she the care the catter the said the said the caster the said the had the was she was she hat she and what she said the said the cat the said the was she her her the said the said the cat the said the she caut the**************Loop write end**************
----- temperature: 0.5
start text
 her her the said the said the cat the said the she caut theend text
**************Loop write start*************
 she hat seres, and the came won' was the say it rear the cat sh

her hear the mouse was on the sarters, and she were alice was a some with a great with a great while was to the sormers to her to do won't she were alice was in the sard to her hear the right some of the some and the more the right here, and the mouse was in a a way on the think, and was on the some a came of the sard to her to do won't she had not some of the think, and the more the right hereed **************Loop write end**************
----- temperature: 0.5
start text
he had not some of the think, and the more the right hereed end text
**************Loop write start*************
to the right hereedpt her talk the queen some off, and the courtruress had she was did down and the sharder the said before the shad on filly sting are.  `i'm on the dormouse in the stor in a good to the groohat when in a little the might to the words and who ear the groohat her head had been to the mouse of the ededed to the mork the look, and the more the what eas the mouth had been to her said **********